<a href="https://colab.research.google.com/github/TanChen168/Week5_PreProcessing/blob/main/PreProcessingExercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

filename = '/content/sample_data/insurance.csv'

df = pd.read_csv(filename)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [3]:
# The target we are trying to predict
y = df['charges']
# The features we will use to make the prediction
X = df.drop(columns = 'charges')

In [4]:
# Import the TTS from sklearn
from sklearn.model_selection import train_test_split
# Train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
X_test

,age,sex,bmi,children,smoker,region
764,45,female,25.175,2,no,northeast
887,36,female,30.020,0,no,northwest
890,64,female,26.885,0,yes,northwest
1293,46,male,25.745,3,no,northwest
259,19,male,31.920,0,yes,northwest
...,...,...,...,...,...,...
342,60,female,27.550,0,no,northeast
308,58,male,34.865,0,no,northeast
1128,34,male,32.800,1,no,southwest
503,19,male,30.250,0,yes,southeast


Numerical - Age, BMI

Ordinal - Children

Nominal - Sex, Smoker, Region

In [73]:
# Imports
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#make numeric selectors
num_selector = make_column_selector(dtype_include='number')

#make categorical and numeric selectors
cat_selector = make_column_selector(dtype_include='object')

#select categorical columns
#cat_data = X_train[cat_selector(X_train)]

#instantiate one hot encoder
# ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
# ohe_encoder.fit(cat_data)
# cat_ohe = ohe_encoder.transform(cat_data)
# cat_ohe

#convert to dataframe, extract new column names from encoder
#set prefixes to original column names
#pd.DataFrame(cat_ohe, columns=ohe_encoder.get_feature_names(cat_data.columns))


In [78]:
from math import remainder
#instantiate preprocessing objects
median_imputer = SimpleImputer(strategy = 'median')
freq_imputer = SimpleImputer(strategy = 'most_frequent')
scaler = StandardScaler()
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# This is the pipeline for numeric columns
num_pipe = make_pipeline(median_imputer, scaler)
# Thius is the pipeline for categorical columns
cat_pipe = make_pipeline(freq_imputer, ohe_encoder)

num_tuple = (num_pipe, num_selector)
cat_tuple = (cat_pipe, cat_selector)

column_transformer = make_column_transformer(num_tuple, cat_tuple)


In [79]:
column_transformer.fit(X_train)

X_train_processed = column_transformer.transform(X_train)
X_test_processed = column_transformer.transform(X_test)

In [80]:
X_train_processed

array([[-1.08716652, -1.14087456, -0.91749963, ...,  1.        ,
         0.        ,  0.        ],
       [-0.80210593, -0.66584152,  0.7436053 , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.83699246,  1.52879447, -0.08694717, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.33584849, -0.8879673 , -0.91749963, ...,  0.        ,
         0.        ,  0.        ],
       [-0.1607196 ,  2.84324666,  0.7436053 , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.12205304, -0.10179179, -0.91749963, ...,  0.        ,
         0.        ,  1.        ]])

In [72]:
from sklearn.linear_model import LinearRegression
# Make a linear regression instance
reg = LinearRegression()
reg.fit(X_train_processed, y_train)
train_score = reg.score(X_train_processed, y_train)
print(train_score)
test_score = reg.score(X_test_processed, y_test)
print(test_score)

0.7449555328228536
0.7672642952734356
